In [17]:
import pandas as pd
import numpy as np

In [8]:
profiles = pd.read_csv('tinder_dataset/profiles_2021-11-10.csv')

/var/folders/yp/2psc2k5d5j52xpn_ds0l8gl40000gn/T/ipykernel_7100/3740928429.py:1: DtypeWarning: Columns (23267) have mixed types. Specify dtype option on import or set low_memory=False.
  profiles = pd.read_csv('tinder_dataset/profiles_2021-11-10.csv')


In [5]:
df = pd.read_json('tinder_dataset/profiles_2021-11-10.json')

print(df.to_string())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOStream.flush timed out


In [ ]:
df

In [10]:
a = list(profiles.columns)

In [12]:
a

['__v',
 '_id',
 'appOpens.2014-11-10',
 'appOpens.2014-11-12',
 'appOpens.2014-11-13',
 'appOpens.2014-11-14',
 'appOpens.2014-11-15',
 'appOpens.2014-11-16',
 'appOpens.2014-11-17',
 'appOpens.2014-11-18',
 'appOpens.2014-11-19',
 'appOpens.2014-11-20',
 'appOpens.2014-11-21',
 'appOpens.2014-11-22',
 'appOpens.2014-11-27',
 'appOpens.2014-11-28',
 'appOpens.2014-11-29',
 'appOpens.2014-11-30',
 'appOpens.2014-12-07',
 'appOpens.2014-12-08',
 'appOpens.2014-12-16',
 'appOpens.2014-12-17',
 'appOpens.2014-12-18',
 'appOpens.2014-12-22',
 'appOpens.2014-12-23',
 'appOpens.2014-12-24',
 'appOpens.2014-12-25',
 'appOpens.2014-12-26',
 'appOpens.2014-12-27',
 'appOpens.2014-12-29',
 'appOpens.2014-12-30',
 'appOpens.2014-12-31',
 'appOpens.2015-01-01',
 'appOpens.2015-01-02',
 'appOpens.2015-01-03',
 'appOpens.2015-01-04',
 'appOpens.2015-01-05',
 'appOpens.2015-01-06',
 'appOpens.2015-01-07',
 'appOpens.2015-01-08',
 'appOpens.2015-01-09',
 'appOpens.2015-01-10',
 'appOpens.2015-01-11',


In [14]:
for element in a:
    print(element)

__v
_id
appOpens.2014-11-10
appOpens.2014-11-12
appOpens.2014-11-13
appOpens.2014-11-14
appOpens.2014-11-15
appOpens.2014-11-16
appOpens.2014-11-17
appOpens.2014-11-18
appOpens.2014-11-19
appOpens.2014-11-20
appOpens.2014-11-21
appOpens.2014-11-22
appOpens.2014-11-27
appOpens.2014-11-28
appOpens.2014-11-29
appOpens.2014-11-30
appOpens.2014-12-07
appOpens.2014-12-08
appOpens.2014-12-16
appOpens.2014-12-17
appOpens.2014-12-18
appOpens.2014-12-22
appOpens.2014-12-23
appOpens.2014-12-24
appOpens.2014-12-25
appOpens.2014-12-26
appOpens.2014-12-27
appOpens.2014-12-29
appOpens.2014-12-30
appOpens.2014-12-31
appOpens.2015-01-01
appOpens.2015-01-02
appOpens.2015-01-03
appOpens.2015-01-04
appOpens.2015-01-05
appOpens.2015-01-06
appOpens.2015-01-07
appOpens.2015-01-08
appOpens.2015-01-09
appOpens.2015-01-10
appOpens.2015-01-11
appOpens.2015-01-12
appOpens.2015-01-13
appOpens.2015-01-14
appOpens.2015-01-15
appOpens.2015-01-16
appOpens.2015-01-17
appOpens.2015-01-18
appOpens.2015-01-19
appOpens.201

In [32]:
for e in profiles["matches.2019-06-01"]: print(e)

nan
0.0
nan
nan
5.0
nan
nan
nan
nan
10.0
nan
nan
nan
nan
nan
nan
2.0
nan
nan
0.0
5.0
nan
nan
2.0
0.0
12.0
0.0
nan
1.0
nan
nan
0.0
nan
3.0
nan
16.0
nan
2.0
11.0
nan
0.0
2.0
nan
nan
nan
nan
nan
10.0
1.0
nan
nan
nan
nan
nan
0.0
nan
nan
3.0
nan
nan
1.0
nan
nan
nan
nan
nan
nan
nan
nan
17.0
nan
nan
1.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
20.0
3.0
nan
nan
nan
nan
26.0
nan
nan
nan
nan
nan
nan
nan
2.0
nan
0.0
nan
nan
0.0
nan
nan
nan
nan
nan
nan
nan
nan
1.0
nan
nan
nan
1.0
0.0
0.0
nan
nan
nan
0.0
31.0
nan
nan
nan
2.0
nan
nan
nan
nan
nan
nan
nan
0.0
nan
nan
nan
0.0
nan
0.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
1.0
nan
nan
0.0
0.0
nan
nan
nan
1.0
nan
0.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.0
nan
nan
2.0
nan
5.0
nan
nan
nan
nan
0.0
nan
nan
13.0
nan
nan
nan
nan
nan
nan
0.0
nan
nan
0.0
nan
0.0
0.0
0.0
nan
1.0
nan
nan
nan
nan
nan
nan
nan
2.0
2.0
nan
1.0
nan
nan
5.0
nan
nan
nan
nan
4.0
nan
nan
nan
nan
nan
nan
nan
1.0
nan
nan
nan
nan
nan
3.0
5.0
nan
4.0
nan
nan
nan
nan
na